# Ch10-2.RNN:단어 자동 완성

### LSTM(Long Short Term Memory)으로 단어 글자 예측하기

#### (예) wor -> d     : word

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#소문자 a~z까지 배열변수에 넣기
char_arr=[chr(ch) for ch in range(97,123)]
num_dic={n:i for i,n in enumerate(char_arr)}
dic_len=len(num_dic)

In [3]:
seq_data=['word','wood','deep','dive','cold','cool','load','love','kiss','kind']

In [4]:
def make_batch(seq_data):
    input_batch=[]
    target_batch=[]
    
    for seq in seq_data:
        input=[num_dic[n] for n in seq[: -1]]
        target=num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [5]:
learning_rate=0.01 #최적화함수에서 사용할 학습률
total_epoch=30#전체 데이터를 학습할 총 횟수
n_step=3
n_hidden=128 #hidden layer의 뉴런 갯수
n_input=n_class=dic_len

In [6]:
X=tf.placeholder(tf.float32,[None,n_step, n_input]) 
Y=tf.placeholder(tf.int32,[None]) 
W=tf.Variable(tf.random_normal([n_hidden, n_class]))
b=tf.Variable(tf.random_normal([n_class]))

In [7]:
cell1=tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1=tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2=tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [8]:
multi_cell=tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [9]:
outputs, states=tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs=tf.transpose(outputs, [1,0,2])
outputs=outputs[-1] #맨 뒤 cell
model=tf.matmul(outputs, W)+b

cost=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    input_batch, target_batch=make_batch(seq_data)
    
    for epoch in range(total_epoch):
        _, loss= sess.run([optimizer,cost], feed_dict={X: input_batch, Y:target_batch})
        print('Epoch: ', '%04d' % (epoch+1), 'AVG_cost= ','{:6f}'.format(loss))
    
    print('완료')
    
    prediction=tf.cast(tf.argmax(model,1),tf.int32)
    prediction_check=tf.equal(prediction,Y)
    accuracy=tf.reduce_mean(tf.cast(prediction_check, tf.float32))
    
    input_batch, target_batch= make_batch(seq_data)
    
    predict, accuracy_val=sess.run([prediction,accuracy],feed_dict={X:input_batch, Y:target_batch})
    
    
    predict_words=[]
    
    for idx, val in enumerate(seq_data):
        last_char=char_arr[predict[idx]]
        predict_words.append(val[:3]+last_char)
    
    print('\n======예측결과 ========')
    print('입력값: ', [w[:3]+' ' for w in seq_data])
    print('예측값: ',predict_words)
    print('정확도: ',accuracy_val)
    

Epoch:  0001 AVG_cost=  3.811745
Epoch:  0002 AVG_cost=  2.880395
Epoch:  0003 AVG_cost=  1.922401
Epoch:  0004 AVG_cost=  1.456142
Epoch:  0005 AVG_cost=  0.608919
Epoch:  0006 AVG_cost=  1.510249
Epoch:  0007 AVG_cost=  0.525544
Epoch:  0008 AVG_cost=  0.420863
Epoch:  0009 AVG_cost=  0.614880
Epoch:  0010 AVG_cost=  0.626983
Epoch:  0011 AVG_cost=  0.210050
Epoch:  0012 AVG_cost=  0.146432
Epoch:  0013 AVG_cost=  0.172914
Epoch:  0014 AVG_cost=  0.268009
Epoch:  0015 AVG_cost=  0.300398
Epoch:  0016 AVG_cost=  0.252014
Epoch:  0017 AVG_cost=  0.076825
Epoch:  0018 AVG_cost=  0.111613
Epoch:  0019 AVG_cost=  0.137420
Epoch:  0020 AVG_cost=  0.143878
Epoch:  0021 AVG_cost=  0.052593
Epoch:  0022 AVG_cost=  0.080637
Epoch:  0023 AVG_cost=  0.069199
Epoch:  0024 AVG_cost=  0.045662
Epoch:  0025 AVG_cost=  0.063003
Epoch:  0026 AVG_cost=  0.040180
Epoch:  0027 AVG_cost=  0.081855
Epoch:  0028 AVG_cost=  0.090404
Epoch:  0029 AVG_cost=  0.018008
Epoch:  0030 AVG_cost=  0.004508
완료

======